In [174]:
import torch
import numpy as np
from copy import deepcopy

## Analyse results

In [175]:
def read_file(kernel_type, dim, benchmark_index, nruns):
    file_name = 'tsp_botorch_'+kernel_type+'_EI_dim_'+str(dim)+'benchmark_index_'+str(benchmark_index)+'_nrun_'+str(nruns)+'.pkl'
    data = torch.load(file_name, weights_only=False)
    l = data['outputs']
    l = [float(i) for i in l]
    return l


def read_file_filename(file_name):
    data = torch.load(file_name, weights_only=False)
    l = data['outputs']
    l = [float(i) for i in l]
    return l


def read_file_no_anchor(kernel_type, dim, benchmark_index, nruns):
    file_name = 'tsp_botorch_'+kernel_type+'_EI_dim_'+str(dim)+'benchmark_index_no_anchor_'+str(benchmark_index)+'_nrun_'+str(nruns)+'.pkl'
    data = torch.load(file_name, weights_only=False)
    l = data['outputs']
    l = [float(i) for i in l]
    return l

In [475]:
import os

def analyse_trial(dim=10, benchmark_index=0, cut=200):
    folders = os.listdir('./results')
    nruns = 20
    results_dict = {}

    for folder in folders:
        if '.' in folder:
            continue
        results_dict[folder] = []
        for nrun in range(nruns):
            results_dict[folder].append(read_file_filename(os.path.join('./results', folder, folder+f'_nrun_{nrun}.pkl')))

    all_results = []
    for key in results_dict.keys():
        results_dict[key] = np.array(results_dict[key])
        all_results.append(results_dict[key][:, :cut])
    # print(all_results[0].shape)
    # return all_results
    global_minimum = np.min(all_results)
    print(global_minimum)
    best_so_far = [np.minimum.accumulate(res, axis=1) for res in all_results]
    regrets = [bfs - global_minimum for bfs in best_so_far]
    for i, key in enumerate(results_dict.keys()):
        results_dict[key] = regrets[i]
        # results_dict[key] = all_results[i]
    return results_dict

In [472]:
import numpy as np
import pandas as pd
from sklearn.metrics import auc

import numpy as np
import pandas as pd
from sklearn.metrics import auc

import numpy as np
import pandas as pd
from sklearn.metrics import auc

def evaluate_algorithms(r: dict, f_opt=None, threshold=None):
    """
    r: dict of {algorithm_name: np.ndarray of shape (n_repeats, n_iterations)}
    f_opt: known global minimum value (float)
    threshold: optional regret threshold to measure how many iterations are needed
    
    Returns:
        pd.DataFrame with aggregated metrics for each algorithm
    """
    results = []

    for algo, regrets in r.items():
        regrets = np.array(regrets)  # shape: (n_repeats, n_iterations)
        n_repeats, n_iterations = regrets.shape

        best_so_far = np.minimum.accumulate(regrets, axis=1)  # shape: (n_repeats, n_iterations)
        final_best = best_so_far[:, -1]
        auc_vals = np.array([
            auc(np.arange(1, n_iterations+1), best_so_far[i])
            for i in range(n_repeats)
        ])

        metrics = {
            "algorithm": algo,
            "final_best_mean": np.mean(final_best),
            "final_best_std": np.std(final_best),
            "auc_best_so_far_mean": np.mean(auc_vals),
            "auc_best_so_far_std": np.std(auc_vals),
        }

        if f_opt is not None:
            simple_regrets = regrets - f_opt  # ✅ 对于最小化，目标值应减去最优值
            cumulative_regrets = np.cumsum(simple_regrets, axis=1)
            mean_simple = np.mean(simple_regrets, axis=1)
            final_simple = simple_regrets[:, -1]
            final_cum = cumulative_regrets[:, -1]

            metrics.update({
                "mean_simple_regret_mean": np.mean(mean_simple),
                "mean_simple_regret_std": np.std(mean_simple),
                "final_simple_regret_mean": np.mean(final_simple),
                "final_simple_regret_std": np.std(final_simple),
                "cumulative_regret_mean": np.mean(final_cum),
                "cumulative_regret_std": np.std(final_cum),
            })
        else:
            metrics.update({
                "mean_simple_regret_mean": None,
                "mean_simple_regret_std": None,
                "final_simple_regret_mean": None,
                "final_simple_regret_std": None,
                "cumulative_regret_mean": None,
                "cumulative_regret_std": None,
            })

        if threshold is not None:
            evals_to_threshold = []
            for i in range(n_repeats):
                for j in range(n_iterations):
                    if best_so_far[i, j] <= threshold:
                        evals_to_threshold.append(j + 1)
                        break
                else:
                    evals_to_threshold.append(n_iterations)
            evals_to_threshold = np.array(evals_to_threshold)
            metrics.update({
                "evals_to_threshold_mean": np.mean(evals_to_threshold),
                "evals_to_threshold_std": np.std(evals_to_threshold),
            })
        else:
            metrics.update({
                "evals_to_threshold_mean": None,
                "evals_to_threshold_std": None,
            })

        results.append(metrics)

    return pd.DataFrame(results)


r = analyse_trial(cut=200)
evaluate_algorithms(r, 329.969018)

329.969018


,algorithm,final_best_mean,final_best_std,auc_best_so_far_mean,auc_best_so_far_std,mean_simple_regret_mean,mean_simple_regret_std,final_simple_regret_mean,final_simple_regret_std,cumulative_regret_mean,cumulative_regret_std,evals_to_threshold_mean,evals_to_threshold_std
0,tsp_botorch_merge_EI_dim_10benchmark_index_whe...,330.961955,1.400853,66348.582463,224.534226,15.541089,7.416205,5.191100,4.771859,3108.217812,1483.241058,None,None
1,tsp_botorch_merge_worst_anchor_EI_dim_10benchm...,330.839984,1.180396,66444.120321,269.130116,19.913313,9.128154,9.940442,12.698160,3982.662545,1825.630789,None,None
2,tsp_botorch_merge_EI_dim_10benchmark_index_k5_...,330.052889,0.174432,66132.302860,146.084482,7.123681,1.101528,5.588660,12.780406,1424.736140,220.305690,None,None
3,tsp_botorch_merge_EI_dim_10benchmark_index_ran...,330.859034,1.150040,66429.986923,331.251267,21.310025,3.659604,18.066461,14.431186,4262.005040,731.920787,None,None
4,tsp_botorch_merge_EI_dim_10benchmark_index_gro...,332.768708,0.982919,66709.759262,302.616530,38.669974,1.023348,31.599276,17.644428,7733.994895,204.669590,None,None
5,tsp_botorch_merge_EI_dim_10benchmark_index_gro...,332.156110,1.257301,66600.545815,296.410273,36.769954,4.478555,34.820200,19.667989,7353.990880,895.710966,None,None
6,tsp_botorch_merge_EI_dim_10benchmark_index_ave...,330.117405,0.245754,66322.670171,247.165137,15.160607,3.448181,2.245157,1.584732,3032.121495,689.636205,None,None
7,tsp_botorch_merge_EI_dim_10benchmark_index_k4_...,330.014179,0.169592,66076.026011,121.981277,6.789502,0.954584,1.329030,1.470682,1357.900358,190.916777,None,None
8,tsp_botorch_merge_EI_dim_10benchmark_index_k6_...,330.078695,0.192578,66161.294014,146.217208,8.523652,1.290559,1.909064,2.109871,1704.730449,258.111701,None,None
9,tsp_botorch_merge_EI_dim_10benchmark_index_k5_...,330.014179,0.073842,66131.611320,139.105395,8.388111,1.017019,4.009847,7.840697,1677.622146,203.403796,None,None


In [528]:
r = analyse_trial()

329.969018


In [529]:
for key in r.keys():
    print(f'{key} mean regret: ', r[key].mean())

tsp_botorch_merge_EI_dim_10benchmark_index_k5_with_permutation_pattern_0 mean regret:  2.4405371559999964
tsp_botorch_merge_EI_dim_10benchmark_index_k4_with_permutation_pattern_0 mean regret:  2.159056133999994
tsp_botorch_merge_EI_dim_10benchmark_index_no_anchor_0 mean regret:  2.7361840119999923
tsp_botorch_merge_EI_dim_10benchmark_index_k3_with_permutation_pattern_0 mean regret:  2.2761458159999948
tsp_botorch_merge_EI_dim_10benchmark_index_k4_permutation_pattern_0 mean regret:  1.9531769419999947
tsp_botorch_bitonic_EI_dim_10benchmark_index_0 mean regret:  5.3665869299999756
tsp_botorch_mallows_EI_dim_10benchmark_index_0 mean regret:  2.239048861999994


In [527]:
r['tsp_botorch_merge_EI_dim_10benchmark_index_k4_with_permutation_pattern_0'][0]

array([33.001712, 33.001712, 22.943312, 22.943312, 22.943312, 15.213584,
       15.213584, 15.213584, 15.213584, 15.213584, 15.213584, 15.213584,
       15.213584, 15.213584, 15.213584, 15.213584, 15.213584, 12.891008,
       12.891008, 12.891008, 12.891008, 12.891008, 12.891008, 12.891008,
        3.741928,  3.741928,  3.741928,  3.741928,  3.741928,  3.741928,
        3.741928,  3.741928,  3.741928,  3.741928,  3.741928,  3.096768,
        3.096768,  3.096768,  3.096768,  1.548384,  1.548384,  1.548384,
        1.548384,  1.548384,  1.548384,  1.548384,  1.548384,  1.548384,
        1.548384,  1.548384,  0.258064,  0.258064,  0.258064,  0.258064,
        0.258064,  0.258064,  0.258064,  0.258064,  0.258064,  0.258064,
        0.258064,  0.258064,  0.258064,  0.258064,  0.258064,  0.258064,
        0.258064,  0.258064,  0.258064,  0.258064,  0.258064,  0.258064,
        0.258064,  0.258064,  0.258064,  0.258064,  0.258064,  0.258064,
        0.258064,  0.258064,  0.258064,  0.258064, 

In [465]:
r['tsp_botorch_merge_EI_dim_10benchmark_index_k5_with_hash_bucket_0'][8]

array([42.531792, 42.531792, 19.078448, 19.078448, 19.078448, 19.078448,
       11.735816, 11.735816, 11.735816, 11.735816, 11.735816, 11.735816,
       11.735816, 11.735816, 11.735816, 11.735816, 11.735816, 11.735816,
       11.735816, 11.735816, 11.735816, 10.187432,  8.12292 ,  8.12292 ,
        5.935472,  3.612896,  3.612896,  3.612896,  3.612896,  2.322576,
        2.322576,  2.322576,  2.322576,  2.322576,  2.322576,  2.322576,
        2.322576,  1.548384,  1.548384,  0.387096,  0.387096,  0.387096,
        0.387096,  0.258064,  0.258064,  0.258064,  0.258064,  0.258064,
        0.258064,  0.258064,  0.258064,  0.258064,  0.258064,  0.258064,
        0.258064,  0.258064,  0.258064,  0.258064,  0.258064,  0.258064,
        0.258064,  0.258064,  0.258064,  0.258064,  0.258064,  0.258064,
        0.258064,  0.258064,  0.258064,  0.129032,  0.129032,  0.129032,
        0.129032,  0.129032,  0.129032,  0.129032,  0.129032,  0.129032,
        0.129032,  0.129032,  0.129032,  0.129032, 

In [467]:
r['tsp_botorch_mallows_EI_dim_10benchmark_index_0'][7]

array([24.1046  , 24.1046  , 24.1046  , 24.1046  ,  7.612888,  7.612888,
        7.612888,  7.612888,  7.612888,  7.612888,  7.612888,  7.612888,
        2.322576,  2.322576,  2.322576,  2.322576,  2.322576,  2.322576,
        2.322576,  2.322576,  2.322576,  2.322576,  1.548384,  1.29032 ,
        1.29032 ,  1.29032 ,  1.29032 ,  1.29032 ,  1.29032 ,  1.29032 ,
        1.29032 ,  1.032256,  1.032256,  1.032256,  1.032256,  1.032256,
        1.032256,  1.032256,  1.032256,  1.032256,  1.032256,  1.032256,
        1.032256,  1.032256,  1.032256,  1.032256,  1.032256,  1.032256,
        1.032256,  1.032256,  1.032256,  1.032256,  1.032256,  1.032256,
        1.032256,  1.032256,  1.032256,  0.64516 ,  0.64516 ,  0.64516 ,
        0.64516 ,  0.516128,  0.516128,  0.258064,  0.258064,  0.258064,
        0.258064,  0.258064,  0.129032,  0.129032,  0.129032,  0.129032,
        0.129032,  0.129032,  0.129032,  0.129032,  0.129032,  0.129032,
        0.129032,  0.129032,  0.129032,  0.129032, 

In [151]:
r['tsp_botorch_bitonic_EI_dim_10benchmark_index_0'].mean()

np.float64(5.3665869299999756)

In [152]:
r['tsp_botorch_mallows_EI_dim_10benchmark_index_0'].mean()

np.float64(2.239048861999994)

In [167]:
r['tsp_botorch_mallows_EI_dim_10benchmark_index_0'][15]

array([68.307712, 56.06796 , 56.06796 , 14.304264, 14.304264, 14.304264,
       14.304264, 14.304264, 14.304264, 14.304264, 12.37488 , 12.37488 ,
       12.37488 , 12.37488 , 12.37488 , 12.37488 , 12.37488 , 12.37488 ,
       12.37488 , 12.37488 , 12.116816, 10.955528, 10.955528, 10.4394  ,
       10.4394  ,  6.18744 ,  1.161288,  0.129032,  0.129032,  0.129032,
        0.129032,  0.129032,  0.129032,  0.129032,  0.129032,  0.129032,
        0.129032,  0.129032,  0.129032,  0.129032,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      , 

In [534]:
from itertools import permutations
import numpy as np

def get_relative_order(window):
    order = sorted(range(len(window)), key=lambda i: window[i])
    relative = [0] * len(window)
    for rank, idx in enumerate(order):
        relative[idx] = rank
    return tuple(relative)

def pattern_count_feature_circular(perm, k):
    """
    计算循环滑动窗口的排序模式计数。
    
    参数：
      - perm: 输入置换（长度为 n 的 list 或 np.array）
      - k: 滑动窗口长度

    返回：
      - 长度为 k! 的 numpy 向量，统计每种模式出现的次数
    """
    n = len(perm)
    patterns = list(permutations(range(k)))
    pattern_to_index = {p: i for i, p in enumerate(patterns)}
    counts = np.zeros(len(patterns), dtype=int)

    for i in range(n):
        window = [perm[(i + j) % n] for j in range(k)]  # 循环索引
        pat = get_relative_order(window)
        idx = pattern_to_index[pat]
        counts[idx] += 1

    return counts

def right_multiply(x, anchor):
    """
    对置换 x 进行右乘 anchor，返回 x ◦ anchor。
    
    参数：
      - x: list[int]，长度为 n 的置换，例如 [2, 0, 1]
      - anchor: list[int]，长度为 n 的置换，例如 [1, 2, 0]
    
    返回：
      - result: list[int]，x 右乘 anchor 的结果
    """
    return [anchor[xi] for xi in x]

In [543]:
perm1 = [3, 1, 4, 2, 0]
perm2 = [2, 4, 1, 0, 3]

k = 3
feat1 = pattern_count_feature_circular(perm1, k)
feat2 = pattern_count_feature_circular(perm2, k)
print(feat1-feat2)

[ 0  1  0 -1  0  0]


In [559]:
anchor = [3, 4, 0, 1, 2]
right1 = right_multiply(perm1, anchor)
right2 = right_multiply(perm2, anchor)
feat1 = pattern_count_feature_circular(right1, k)
feat2 = pattern_count_feature_circular(right2, k)
print(feat1-feat2)

[-1  1  1  0  0 -1]


In [566]:
perm = [3, 1, 4, 2, 0]
k =3 

n = len(perm)
patterns = list(permutations(range(k)))
pattern_to_index = {p: i for i, p in enumerate(patterns)}
counts = np.zeros(len(patterns), dtype=int)

for i in range(n):
    window = [perm[(i + j) % n] for j in range(k)]  # 循环索引
    print(window)
    pat = get_relative_order(window)
    idx = pattern_to_index[pat]
    counts[idx] += 1

[3, 1, 4]
[1, 4, 2]
[4, 2, 0]
[2, 0, 3]
[0, 3, 1]


In [565]:
patterns

[(0, 1, 2), (0, 2, 1), (1, 0, 2), (1, 2, 0), (2, 0, 1), (2, 1, 0)]

In [560]:
right1

[1, 4, 2, 0, 3]

In [556]:
right2

[0, 2, 4, 3, 1]